In [16]:
from tasks import run_md5
from tasks import run_fastqc
from invoke import run
import time
from datetime import datetime
from core.utils import Tibanna
from core import ff_utils

# for a given experiment set and some parameters like instrument
# print set of files and their partA hic workflow status
# if there are one that are running report the number of running cases
# if there are file pairs that don't have a corresponding part A, report them separately


wf_md5 = "md5"
wf_fastqc = "fastqc-0-11-4-1/1"

env = 'fourfront-webdev'
tibanna = Tibanna(env=env)
run_md_qc = raw_input("Do you wanna run md5 and/or fastqc if missing? (md5/qc/all/none)")
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)

def summarize_file(fastq_file_id):
    file_resp = ff_utils.get_metadata(fastq_file_id, connection=ff, frame = 'embedded')
    qc = False
    file_id = file_resp['accession']
    sequencer = file_resp.get('instrument')
    relations = file_resp.get('related_files')
    status = file_resp.get('status')
    workflows = file_resp.get('workflow_run_inputs')
    first_alias = file_resp.get('aliases',[None])[0]
    pair_no = file_resp.get('paired_end')
    # get related file
    paired_file = ''
    for relation in relations:
        if relation['relationship_type'] == 'paired with':
            paired_file = relation['file']['accession']
    
    # is there a qc?
    if file_resp.get('quality_metric'):
        qc = True
    # Check workflows for qc fastqc workflow partA
    last_part_A = ''
    last_part_A_status = 'did_not_run'
    md5_status = 'did_not_run'
    fastqc_status = 'did_not_run'
    # Assumes workflow_runs come in time ordered list, and grabs the last ones for each wf run
    if workflows:
        for wfr_resp in workflows:
            wfr_name = wfr_resp['display_title']
            if wfr_name.startswith(wf_md5):
                md5_status = wfr_resp.get('run_status')     
            elif wfr_name.startswith(wf_fastqc):
                fastqc_status = wfr_resp.get('run_status')     
#             elif wfr_name.startswith(wf_partA):
#                 last_part_A=wfr_resp['uuid']
#                 last_part_A_status = wfr_resp.get('run_status')  
                
    # Check for md5 and fastqc, and if not complete, run or report it. 
    # check if md5 step is completed properly

    if status != "uploaded" or md5_status != 'complete':
        # if not, shall we run it?
        if run_md_qc in ['md5', 'all']:
            print 'md5 running for', file_resp['accession']
            code_md5= "invoke run_md5 " + env + " " + file_resp['accession'] + " " + file_resp['uuid']
            run(code_md5)
            print ''
            time.sleep(10)
        # user does not want it to be run, so just report
        else:
            print 'md5 run missing for', file_resp['accession']
    # check fastqc if md5 is fine
    else:
        if not qc or fastqc_status != 'complete':
            # if not, shall we run it?
            if run_md_qc in ['qc', 'all']:
                print 'fastqc running for', file_resp['accession']
                code_qc= "invoke run_fastqc " + env + " " + file_resp['accession'] + " " + file_resp['uuid']
                run(code_qc)
                print ''    
                time.sleep(10)
            # user does not want it to be run, so just report
            else:
                print 'fastqc run missing for', file_resp['accession'], fastqc_status
                print 


Do you wanna run md5 and/or fastqc if missing? (md5/qc/all/none)all


In [17]:
delet = 0
no_f = 0
no_s3 = 0
miseq = 0
all_files = ff_utils.get_metadata('files-fastq', connection=ff)['@graph']

for fastq_file in all_files:
    if fastq_file['status'] in ['deleted', 'uploading', 'upload failed']:
        delet += 1
        continue
    #if no uploaded file in the file item report and skip
    if not fastq_file.get('filename'):
        print fastq_file['accession'], "does not have a file"
        no_f += 1
        continue
    # check if file is in s3
    if not tibanna.s3.does_key_exist(fastq_file['upload_key'], tibanna.s3.raw_file_bucket):
        print fastq_file['accession'], "does not have a file in S3"
        no_s3 += 1
        continue 
    # skip miseq for fastqc
    if fastq_file.get('instrument') == "Illumina MiSeq":
        miseq += 1
        continue
    file_info = summarize_file(fastq_file['uuid'])
    
print str(delet), 'deleted files skipped'
print str(no_f), 'files with no uploads skipped'
print str(no_s3), 'files with missing s3 file skipped'
print str(miseq), 'files skipped because they are miseq'

fastqc running for 4DNFI6BFAKQI
about to start run fastqc-0-11-4-1-1_4DNFI6BFAKQI.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2017, 6, 30, 11, 58, 34, 99000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': 'f85d4741-5dac-11e7-8914-d7bae0f11e5e', 'HTTPHeaders': {'x-amzn-requestid': 'f85d4741-5dac-11e7-8914-d7bae0f11e5e', 'content-length': '154', 'content-type': 'application/x-amz-json-1.0'}}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:run_sbg_workflow_3:fastqc-0-11-4-1-1_4DNFI6BFAKQI.fastq.gz'}
url to view status:
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:execution:run_sbg_workflow_3:fastqc-0-11-4-1-1_4DNFI6BFAKQI.fastq.gz

fastqc running for 4DNFI65A88XH
about to start run fastqc-0-11-4-1-1_4DNFI65A88XH.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2017, 6, 30, 11, 58, 45, 489000, tzinfo=tzlocal()


fastqc running for 4DNFIPSKHIBT
about to start run fastqc-0-11-4-1-1_4DNFIPSKHIBT.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2017, 6, 30, 12, 0, 40, 342000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': '439c1f87-5dad-11e7-837b-b30d2f5f58c7', 'HTTPHeaders': {'x-amzn-requestid': '439c1f87-5dad-11e7-837b-b30d2f5f58c7', 'content-length': '154', 'content-type': 'application/x-amz-json-1.0'}}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:run_sbg_workflow_3:fastqc-0-11-4-1-1_4DNFIPSKHIBT.fastq.gz'}
url to view status:
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:execution:run_sbg_workflow_3:fastqc-0-11-4-1-1_4DNFIPSKHIBT.fastq.gz

fastqc running for 4DNFIB1JOP3F
about to start run fastqc-0-11-4-1-1_4DNFIB1JOP3F.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2017, 6, 30, 12, 0, 51, 909000, tzinfo=tzlocal()


fastqc running for 4DNFIF5LBBWQ
about to start run fastqc-0-11-4-1-1_4DNFIF5LBBWQ.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2017, 6, 30, 12, 2, 47, 397000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': '8f583fa3-5dad-11e7-81d0-6da9104ab671', 'HTTPHeaders': {'x-amzn-requestid': '8f583fa3-5dad-11e7-81d0-6da9104ab671', 'content-length': '154', 'content-type': 'application/x-amz-json-1.0'}}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:run_sbg_workflow_3:fastqc-0-11-4-1-1_4DNFIF5LBBWQ.fastq.gz'}
url to view status:
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:execution:run_sbg_workflow_3:fastqc-0-11-4-1-1_4DNFIF5LBBWQ.fastq.gz

fastqc running for 4DNFIDKRWIPN
about to start run fastqc-0-11-4-1-1_4DNFIDKRWIPN.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2017, 6, 30, 12, 2, 58, 912000, tzinfo=tzlocal()


md5 running for 4DNFIX3X22EQ
about to start run md5_4DNFIX3X22EQ.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2017, 6, 30, 12, 4, 49, 642000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': 'd834be8c-5dad-11e7-92a3-e5af9f9ac94e', 'HTTPHeaders': {'x-amzn-requestid': 'd834be8c-5dad-11e7-92a3-e5af9f9ac94e', 'content-length': '140', 'content-type': 'application/x-amz-json-1.0'}}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:run_sbg_workflow_3:md5_4DNFIX3X22EQ.fastq.gz'}
url to view status:
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:execution:run_sbg_workflow_3:md5_4DNFIX3X22EQ.fastq.gz

md5 running for 4DNFIBF377E2
about to start run md5_4DNFIBF377E2.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2017, 6, 30, 12, 5, 1, 484000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 2


fastqc running for 4DNFILT5X7TK
about to start run fastqc-0-11-4-1-1_4DNFILT5X7TK.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2017, 6, 30, 12, 6, 59, 670000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': '25b79472-5dae-11e7-82ae-d58afe66e739', 'HTTPHeaders': {'x-amzn-requestid': '25b79472-5dae-11e7-82ae-d58afe66e739', 'content-length': '153', 'content-type': 'application/x-amz-json-1.0'}}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:run_sbg_workflow_3:fastqc-0-11-4-1-1_4DNFILT5X7TK.fastq.gz'}
url to view status:
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:execution:run_sbg_workflow_3:fastqc-0-11-4-1-1_4DNFILT5X7TK.fastq.gz

fastqc running for 4DNFI23CNILG
about to start run fastqc-0-11-4-1-1_4DNFI23CNILG.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2017, 6, 30, 12, 7, 11, 173000, tzinfo=tzlocal()

...


fastqc running for 4DNFI2QMTOOK
about to start run fastqc-0-11-4-1-1_4DNFI2QMTOOK.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2017, 6, 30, 13, 37, 31, 697000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': 'cb73499b-5dba-11e7-a035-f7358fb12262', 'HTTPHeaders': {'x-amzn-requestid': 'cb73499b-5dba-11e7-a035-f7358fb12262', 'content-length': '154', 'content-type': 'application/x-amz-json-1.0'}}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:run_sbg_workflow_3:fastqc-0-11-4-1-1_4DNFI2QMTOOK.fastq.gz'}
url to view status:
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:execution:run_sbg_workflow_3:fastqc-0-11-4-1-1_4DNFI2QMTOOK.fastq.gz

fastqc running for 4DNFIKB7MZ71
about to start run fastqc-0-11-4-1-1_4DNFIKB7MZ71.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2017, 6, 30, 13, 37, 44, 132000, tzinfo=tzlocal


fastqc running for 4DNFIBX1H53B
about to start run fastqc-0-11-4-1-1_4DNFIBX1H53B.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2017, 6, 30, 13, 39, 41, 248000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': '18adcc5c-5dbb-11e7-9529-49f2b8e45e20', 'HTTPHeaders': {'x-amzn-requestid': '18adcc5c-5dbb-11e7-9529-49f2b8e45e20', 'content-length': '154', 'content-type': 'application/x-amz-json-1.0'}}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:run_sbg_workflow_3:fastqc-0-11-4-1-1_4DNFIBX1H53B.fastq.gz'}
url to view status:
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:execution:run_sbg_workflow_3:fastqc-0-11-4-1-1_4DNFIBX1H53B.fastq.gz

fastqc running for 4DNFINVSBKD2
about to start run fastqc-0-11-4-1-1_4DNFINVSBKD2.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2017, 6, 30, 13, 39, 52, 738000, tzinfo=tzlocal


fastqc running for 4DNFI5WFZER4
about to start run fastqc-0-11-4-1-1_4DNFI5WFZER4.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2017, 6, 30, 13, 41, 49, 950000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': '65622e5e-5dbb-11e7-ac62-676ae2c974f7', 'HTTPHeaders': {'x-amzn-requestid': '65622e5e-5dbb-11e7-ac62-676ae2c974f7', 'content-length': '153', 'content-type': 'application/x-amz-json-1.0'}}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:run_sbg_workflow_3:fastqc-0-11-4-1-1_4DNFI5WFZER4.fastq.gz'}
url to view status:
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:execution:run_sbg_workflow_3:fastqc-0-11-4-1-1_4DNFI5WFZER4.fastq.gz

fastqc running for 4DNFIK8LPUSR
about to start run fastqc-0-11-4-1-1_4DNFIK8LPUSR.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2017, 6, 30, 13, 42, 2, 894000, tzinfo=tzlocal(


fastqc running for 4DNFIZKNQNPM
about to start run fastqc-0-11-4-1-1_4DNFIZKNQNPM.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2017, 6, 30, 13, 43, 59, 544000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': 'b2a14444-5dbb-11e7-b3b9-299c48472fe8', 'HTTPHeaders': {'x-amzn-requestid': 'b2a14444-5dbb-11e7-b3b9-299c48472fe8', 'content-length': '154', 'content-type': 'application/x-amz-json-1.0'}}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:run_sbg_workflow_3:fastqc-0-11-4-1-1_4DNFIZKNQNPM.fastq.gz'}
url to view status:
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:execution:run_sbg_workflow_3:fastqc-0-11-4-1-1_4DNFIZKNQNPM.fastq.gz

fastqc running for 4DNFIGMSWSOC
about to start run fastqc-0-11-4-1-1_4DNFIGMSWSOC.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2017, 6, 30, 13, 44, 12, 199000, tzinfo=tzlocal


fastqc running for 4DNFIL1W2A7V
about to start run fastqc-0-11-4-1-1_4DNFIL1W2A7V.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2017, 6, 30, 13, 46, 7, 229000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': 'febb88e6-5dbb-11e7-81d0-6da9104ab671', 'HTTPHeaders': {'x-amzn-requestid': 'febb88e6-5dbb-11e7-81d0-6da9104ab671', 'content-length': '154', 'content-type': 'application/x-amz-json-1.0'}}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:run_sbg_workflow_3:fastqc-0-11-4-1-1_4DNFIL1W2A7V.fastq.gz'}
url to view status:
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:execution:run_sbg_workflow_3:fastqc-0-11-4-1-1_4DNFIL1W2A7V.fastq.gz

fastqc running for 4DNFIFLPFGCP
about to start run fastqc-0-11-4-1-1_4DNFIFLPFGCP.fastq.gz
response from aws was: 
 {u'startDate': datetime.datetime(2017, 6, 30, 13, 46, 18, 660000, tzinfo=tzlocal(

In [ ]:
all_sets = ['dciclab:rao_rep07',
'dciclab:rao_rep02',
'dciclab:rao_rep12',
'dciclab:rao_rep13',
'dcic:Selvaraj_gm12878_hic',
'dcic:Jin_imr90_hic']

for my_rep_set in all_setsXXXX:
    print my_rep_set
    rep_resp = ff_utils.get_metadata(my_rep_set, connection=ff)['experiments_in_set']   
    for exp in rep_resp:
        exp_resp = ff_utils.get_metadata(exp, connection=ff)
        exp_files = exp_resp['files']
        for fastq_file in exp_files:
            file_resp = ff_utils.get_metadata(fastq_file, connection=ff, frame = 'embedded')  
            #if no uploaded file in the file item report and skip
            if not file_resp.get('filename'):
                print file_resp['accession'], "does not have a file"
                continue
            # check if file is in s3
            if not tibanna.s3.does_key_exist(file_resp['upload_key'], tibanna.s3.raw_file_bucket):
                print file_resp['accession'], "does not have a file in S3"
                continue
            file_info = summarize_file(file_resp)
    print
